# `Object detection using Mask-RCNN`
## `Mask-RCNN on custom dataset` (tough one)
### It also predict the region of the object

In [1]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib

# root directory of the project
ROOT_DIR = r'C:\Users\manuj\DL_Concepts\CNN\Mask_RCNN'

DEFAULT_LOGS_DIR =os.path.join(ROOT_DIR,'logs')

MODEL_DIR = os.path.join(ROOT_DIR,'\model')

WEIGHTS_PATH = r'C:\Users\manuj\DL_Concepts\CNN\Mask_RCNN\model\mask_rcnn_coco.h5'

AttributeError: module 'keras.engine' has no attribute 'Layer'

In [ ]:
class CustomConfig(Config):
    NAME = 'object'
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 3 # background + phone + laptop + tablet
    STEPS_PER_EPOCH = 10
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
config = CustomConfig()
CUSTOM_DIR = os.path.join(ROOT_DIR,'/dataset/')

class InferenceConfig(config.__class__):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.7

config = InferenceConfig()
config.display()

In [ ]:
class CustomDataset(utils.Dataset):
    def load_custom(self,dataset_dir,subset):
        self.add_class('object',1,'laptop')
        self.add_class('object',2,'tablet')
        self.add_class('object',1,'mobile')
        
        assert subset in ['train','val']
        dataset_dir = os.path.join(dataset_dir,subset)
        annotation1 = json.load(open(r'C:\Users\manuj\DL_Concepts\CNN\Mask_RCNN\project.json'))
        
        annotations = list(annotation1.values())
        
        annotations = [a for a in annotations if a['regions']]
        
        ## add images
        
        for a in annotations:
            
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['names'] for s in a['regions']]
            print('objects:',objects)
            name_dict = {'laptop':1,'tablet':2,'mobile':3}
            num_ids = [name_dict[a] for a in objects]
            
            print('numids',num_ids)
            image_path = os.path.join(dataset_dir,a['filename'])
            image = skimage.io.imread(image_path)
            
            # usng opencv, please convert from BGR to RGB
            height,width = image.shape[:2]
            
            self.add_image(
            'object',
            image_id = a['filename'],
            path = image_path,
            width = width , height = height,
            polygons = polygons,
            num_ids = num_ids
            )
            
    def load_mask(self,image_id):
        image_info = self.image_info[image_id]
        if image_info['source'] != 'object':
            return super(self.__class__.self).load_mask(image_id)
        
        info = self.image_info[image_id]
        if info['source'] != 'object':
            return super(self.__class__.self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info['height'],info['width'],len(info['polygons'])],
                       dtype=np.uint8)
        
        for i,p in enumerate(info['polygons']):
            rr,cc = skimage.draw.polygon(p['all_points_y'],p['all_points_x'])
            
            mask[rr,cc,i] = 1
            
            num_ids = np.array(num_ids,dtype=np.int32)
            return mask,num_ids
        
    def image_reference(self,image_id):
        # return the path of hte image
        info = self.image_info[image_id]
        if info['source'] == 'object':
            return info['path']
        else:
            super(self.__class__.self).image_reference(image_id)
        

In [ ]:
# Inspect the model in training or inference modes calues: 'inference','training'

TEST_MODE = 'inference'
ROOT_DIR = r'C:\Users\manuj\DL_Concepts\CNN\Mask_RCNN'

def get_ax(rows=1,cols=1,size=16):
    _,ax = plt.subplots(rows,cols,figsize=(size*cols,size*rows))
    return ax

# load validation dataset
# must call before using the dataset

CUSTOM_DIR = r'C:\Users\manuj\DL_Concepts\CNN\Mask_RCNN\dataset'
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR,'val')
dataset.prepare
print('Images: {}\n Classes: {}'.format(len(dataset.image_ids),dataset.class_names))

## `END -----------------------------------------------`